## Assignment 4:CNN with Applications
### Adithya Jayan - 181EC102 | Anvith M - 181EC105

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tqdm.notebook as tqdm
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten, AvgPool2D
from tensorflow.keras.optimizers import Adam
from tqdm.keras import TqdmCallback
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
keras.backend.clear_session()

##### Q.1.
- (i) Write python code for plotting the following activation functions and their derivative for the input x in the range of -20 to 20. 
    - (a) ReLU 
    - (b) LekayReLU 
    - (c) Parametric ReLU 
    - (d) Exponential ReLU(ELU) 
    - (e) Scaled Exponential Linear Units (SELU) 
    - (f) SoftPlus (Smooth ReLU) 

In [ ]:
##Defining required functions for part 1.(i)

def plot_relu(Input,plot=1):
    output = Input.copy()
    output[output<0] = 0
    
    derivative = np.ones_like(Input)
    derivative[Input<0] = 0
        
    if(plot):
        plt.figure()
        plt.title("ReLU plot")
        plt.plot(Input,output)
        
        plt.figure()
        plt.title("ReLU derivative plot")
        plt.plot(Input,derivative)
    return(output)

def plot_leaky_relu(Input,plot=1):
    output = Input.copy()
    derivative = np.ones_like(Input)
    for i in range(len(Input)):
        if(Input[i]<0): 
            output[i]=Input[i]*0.01
            derivative[i] = 0.01
    if(plot):
        plt.figure()
        plt.title("Leaky ReLU plot")
        plt.plot(Input,output)
        
        plt.figure()
        plt.title("Leaky ReLU derivative plot")
        plt.plot(Input,derivative)
    return(output)

def plot_Parametric_relu(Input,alpha,plot=1):
    output = Input.copy()
    derivative = np.ones_like(Input)
    for i in range(len(Input)):
        if(Input[i]<0): 
            output[i]=Input[i]*alpha
            derivative[i] = alpha
    if(plot):
        plt.figure()
        plt.title("Parametric ReLU plot")
        plt.plot(Input,output)
        
        plt.figure()
        plt.title("Parametric ReLU derivative plot")
        plt.plot(Input,derivative)
    return(output)

def plot_Exponential_relu(Input,alpha,plot=1):
    output = Input.copy()
    derivative = np.ones_like(Input)
    for i in range(len(Input)):
        if(Input[i]<0): 
            output[i]=(np.exp(Input[i])-1)*alpha
            derivative[i] = output[i] + alpha
    if(plot):
        plt.figure()
        plt.title("ELU plot")
        plt.plot(Input,output)
        
        plt.figure()
        plt.title("ELU derivative plot")
        plt.plot(Input,derivative)
    return(output)

def plot_Scaled_Exponential_relu(Input,alpha,lamda,plot=1):
    output = Input.copy()
    derivative = np.ones_like(Input)
    for i in range(len(Input)):
        if(Input[i]<0): 
            output[i]=(np.exp(Input[i])-1)*alpha
            derivative[i] = (np.exp(Input[i]))*alpha
    output = output*lamda
    if(plot):
        plt.figure()
        plt.title("Scaled ELU plot")
        plt.plot(Input,output)
        
        plt.figure()
        plt.title("Scaled ELU derivative plot")
        plt.plot(Input,derivative)
    return(output)

def plot_Smooth_relu(Input,plot=1):
    output = np.log(1+np.exp(Input))
    derivative = 1/(1+np.exp(-1*Input))
    if(plot):
        plt.figure()
        plt.title("SoftPlus plot")
        plt.plot(Input,output)
        
        plt.figure()
        plt.title("SoftPlus derivative plot")
        plt.plot(Input,derivative)
    return(output)

In [ ]:
#Solution for 1.(i)

#Defining Input range
Input = np.linspace(-20,20,100)

#Defining required parameters
alpha = 0.1
lamda = 0.5

#Calculation and plotting via function call
plot_relu(Input,plot=1);
plot_leaky_relu(Input,plot=1);
plot_Parametric_relu(Input,alpha,plot=1);
plot_Exponential_relu(Input,alpha,plot=1);
plot_Scaled_Exponential_relu(Input,alpha,lamda,plot=1);
plot_Smooth_relu(Input,plot=1);

##### Q1.
- (ii)
    - (a).Write python from scratch for 2D Linear convolution between input=np.array([[1,2,3],[4,5,6],[7,8,9]])and filter=np.array([[1,2,1],[0,0,0],[-1,-2,-1]]) 
    - (b).Write python from scratch for 2D Linear convolution by Toeplitz matrix method between input image(lena.jpg) and kernel = np.array([[1, 2, 1],[2, 4, 2],[1, 2, 1]]))/16 
    - (c) Compute number of multiplications and parameters required for 2D Linear Convolution in part (a) and part(b) 
    - (d) Apply Max pooling and Average pooling on convoled image in part (b) 

In [ ]:
#Functions for part 1.(ii).(a)

def convolve(Image,kernel,padding = (0,0),s = 1):
    
    kernel = np.flipud(np.fliplr(kernel))
    p1,p2 = padding
    
    x,y = Image.shape

    x_k,y_k = kernel.shape

    out_x = (x-x_k+2*p1)//s+1
    out_y = (y-y_k+2*p2)//s+1
    
    out = np.zeros((out_x,out_y))
    
    Img_padded = np.zeros((x+2*p1,y+2*p2))
    Img_padded[p1:p1+x,p2:p2+y] = Image
    Image = Img_padded
    
    for i in range(out_x):
        i_img = i*s
        for j in range(out_y):
            j_img = j*s
            out[i,j] = np.sum(Image[i_img:i_img+x_k,j_img:j_img+y_k]*kernel)
            
    print("Convolution:\n",out)
    print("\nTotal number of multiplications is: ",(out_x*out_y*x_k*y_k))
    
    return out

In [ ]:
#Solution for part 1.(ii).(a)

Image = np.array([[1,2,3],[4,5,6],[7,8,9]])
print("Image:\n",Image)

kernel = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
print("Kernel:\n",kernel)

convolved = convolve(Image,kernel,(1,1))


In [ ]:
#Functions for part 1.(ii).(b) and 1.(ii).(c)
def rightrotate(arr):
    l = len(arr)
    y = arr[l-1]
    for n in range(l-1,-1,-1):
        arr[n] = arr[n-1]
    arr[n] = y
    return arr

def convolve2dmatrix(input,kernel):
    r1 = np.shape(input)[0]
    c1 = np.shape(input)[1]
    r2 = np.shape(kernel)[0]
    c2 = np.shape(kernel)[1]

    #dimensions of output
    r3 = r1+r2-1
    c3 = c1+c1-1

    #zero padding kernel
    kernel = np.pad(kernel,((0,r3-r2),(0,c3-c2)), 'constant')

    #Creating toeplitz submatrices
    mylist = []
    for i in range(r3):
        x = kernel[i]
        for j in range(c1):
            if (j==0):
                temp = np.array([x])
            else:
                x = rightrotate(x)
                temp = np.vstack((temp,x))
        mylist.append(temp)
  
    #Creating the Blocked Toeplitz matrix
    #soft coded for general case
    templist = mylist
    length = len(templist)
    for j in tqdm.tqdm(range(c1),leave=False,desc = "Convolution progress"):
        if (j==0):
            for k in range(length):
                if (k==0):
                    H = np.block([templist[0]])
                else:
                    H = np.block([H,templist[k]])
            Hfinal = np.block(H)
        else:
            templist = rightrotate(templist)
            for k in range(length):
                if (k==0):
                    H = np.block([templist[0]])
                else:
                    H = np.block([H,templist[k]])
            Hfinal = np.block([[Hfinal],[H]])
    Hfinal = np.transpose(Hfinal)

    #reshape input matrix
    input = np.reshape(input,r1*c1)

    #multiply both
    result_vector = np.matmul(Hfinal, input)

    #reshape output vector
    result_vector = np.reshape(result_vector,(r3,c3))
    
    crop = result_vector[0:102, 0:102]
    plt.imshow(crop,cmap='gray')
    plt.title("Convolved Image:")
    plt.show()
    print("")
    
    return crop
    
def computemultiplications(img,kernel):
    Dfr = np.shape(img)[0]
    Dfc = np.shape(img)[1]
    Dkr = np.shape(kernel)[0]
    Dkc = np.shape(kernel)[1]

    #dimensions of output
    Dpr = Dfr+Dkr-1
    Dpc = Dfc+Dkc-1

    total = (Dkr**2)*(Dpr**2)
    return total

In [ ]:
#Solution for part 1.(ii).(b) and 1.(ii).(c)

input2 = cv2.imread('lena.tif',0)
input2 = cv2.resize(input2,(100,100))
kernel2 = np.array([[1,2,1],[2,4,2],[1,2,1]])
kernel2 = kernel2/16

plt.imshow(input2,cmap='gray')
plt.title("Image:")
plt.show()
print("Kernel:\n",kernel2)
print("")

#Converting image to numpy array
input2 = np.array(input2)

#Calling Convolve2dmatrix
convolved = convolve2dmatrix(input2,kernel2)

#Compute Multiplications
print("The total number of multiplications are:",computemultiplications(input2,kernel2))

In [ ]:
#Functions for part 1.(ii).(d)

def Max_pooling(Image,kernel_size):
    
    x,y = Image.shape

    x_k,y_k = kernel_size

    out_x = (x-x_k)+1
    out_y = (y-y_k)+1
    
    out = np.zeros((out_x,out_y))
        
    for i in range(out_x):
        for j in range(out_y):
            out[i,j] = np.max(Image[i:i+x_k,j:j+y_k])
            
    plt.imshow(out,cmap='gray')
    plt.title("Max_pooled Image:")
    plt.show()        
    
#Functions for part (d)

def Average_pooling(Image,kernel_size):
    
    x,y = Image.shape

    x_k,y_k = kernel_size

    out_x = (x-x_k)+1
    out_y = (y-y_k)+1
    
    out = np.zeros((out_x,out_y))
        
    for i in range(out_x):
        for j in range(out_y):
            out[i,j] = np.mean(Image[i:i+x_k,j:j+y_k])
            
    plt.imshow(out,cmap='gray')
    plt.title("Average_pooled Image:")
    plt.show()     

In [ ]:
#Solution for part 1.(ii).(d)

Max_pooling(convolved,(3,3))
Average_pooling(convolved,(3,3))

##### Q1.
- (iii) 
    - (a)Write python from scratch for 2D Spatial Separbale convolution between input image(lena.jpg) and Gausian filter = np.array([1,4,6,4,1],[4,16,24,16,4],[6,24,36,24,6],[4,16,24,16,4],[1,4,6,4,1])/256 
    - (b) Compute number of multiplications and parameters required for2D Spatial Separbale convolution 

In [ ]:
#Define Inputs
Image = cv2.imread('lena.tif',0) #Image
Image = cv2.resize(Image,(100,100)) 
Image = np.array(Image)

plt.imshow(Image,cmap='gray')
plt.title("Input Image")

In [ ]:
filter1 = np.array([[1,4,6,4,1]])
filter2 = np.reshape(filter1,(5,1)) #Column filter

In [ ]:
inter_image = convolve(Image,filter2) #Intermediate Image
plt.imshow(inter_image,cmap='gray')
plt.title("Intermediate Image")

In [ ]:
final_image = convolve(inter_image,filter1) #final Output

plt.imshow(final_image,cmap='gray')
plt.title("Final Image")

    
##### Q. 2. Build a CNN(LeNet5) model from scratch to recognize handwritten digit from the optical handwritten digit dataset(Use Sklearn Dataset and split dataset into training dataset (80%) and testing dataset (20%) ). Use a Stochastic gradient descent algorithm to learn model with parameters for α = 0.01 and random parameters of the CNN model for 
- (a) Softmax loss function 
- (b) Focal loss function.
    - (i) Plot a comparative loss curve for at least 200 epochs. 
    - (ii) Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy on test datset and ROC curve for each loss function. 
    - (iii) Repeat part (i) to (ii) using an Adam gradient descent algorithm 
    - (iv) Implement above CNN model with Keras/Tensorflow/Pytorch Library and verify the above results. 

In [ ]:
#Function for debugging
def check_nan(Layer_name,**variables):
    if any(np.isnan(val).any() for val in variables.values()):
        print("************* nan detected at {}! *************\n".format(layer_name))
        for key,value in variables.items():
            print("Variable ",key,":")
            print(value,"\n")

## Required layers

# Conv2D
class conv2d_layer:
    def __init__(self,shape,n,anything_else = None):
        self.w = np.random.rand(n,shape[0],shape[1],shape[2])/(np.product(shape)+1)
        self.b = np.random.rand(n)/(np.product(shape)+1)
        self.n = n
        
    def forward(self,Image):    
        
        if(len(Image.shape)==2):
            Image=np.reshape(Image,(1,Image.shape[0],Image.shape[1]))
            
        x,y = Image[0].shape

        _ , x_k , y_k = self.w[0].shape

        out_x = x-x_k+1
        out_y = y-y_k+1
        
        output = []
        
        for pos in range(self.n):
            out = np.zeros((out_x,out_y))
            
            for i in range(out_x):
                for j in range(out_y):
                    out[i,j] = np.sum(Image[:,i:i+x_k,j:j+y_k]*self.w[pos]) + self.b[pos]
            output = output + [out]
        output = np.array(output)      
            
#         check_nan(Layer_name = "conv2d_layer forward step",
#                   output = output,
#                   Image = Image,
#                   self_Weight = self.w,
#                   self_Bias = self.b)
        
        return output
    
    def reverse(self,Image,Errors):
        
        if(len(Image.shape)==2):
            Image=np.reshape(Image,(1,Image.shape[0],Image.shape[1]))
            
        x,y = Image[0].shape
        _ ,x_k,y_k = self.w[0].shape
        err_x,err_y = Errors[0].shape
        
        output = np.zeros_like(Image,Errors.dtype)
        for layer in range(self.n):
            for i in range(err_x):
                for j in range(err_y):
                    output[:,i:i+x_k,j:j+y_k] += Errors[layer,i,j]*self.w[layer]
                    self.w[layer] = self.w[layer] - Errors[layer,i,j]*Image[:,i:i+x_k,j:j+y_k]
                    self.b[layer] = self.b[layer] - Errors[layer,i,j]
                
#         check_nan(Layer_name = "conv2d_layer reverse step",
#                   output = output,
#                   Image = Image,
#                   Errors = Errors,
#                   Weights = self.w,
#                   Bias = self.b)
        
        return output
    
    def display(self):
        print("\n**********Convolution Parameters**********")
        print("Weights are:\n", self.w)
        print("Bias is:\n ", self.b)
        print("#Kernels is:\n ", self.n)
        
# tanh
class tanh:
    
    def forward(self,Input):
        exp = np.exp(-2*Input)
        output = (1-exp)/(1+exp)
        
#         check_nan(Layer_name = "tanh forward step",
#           Input = Input,
#           output = output)
            
        return output
    
    def reverse(self,Input,Errors):
        der = 1-np.square(self.forward(Input))
        output = der*Errors
        
#         check_nan(Layer_name = "tanh reverse step",
#                   Input = Input,
#                   Errors = Errors,
#                   output = output,
#                   der = der)
        
        return output
    
# AveragePooling2D
class AveragePooling2D:
    
    def __init__(self,shape,stride,anything_else = None):
        self.shape = shape
        self.stride = stride
        
    def forward(self,Image):  
        if(len(Image.shape)==2):
            Image=np.reshape(Image,(1,Image.shape[0],Image.shape[1]))
            
        x,y = Image[0].shape
        s=self.stride
        x_k,y_k = self.shape
        
        out_x = (x-x_k)//s+1
        out_y = (y-y_k)//s+1

        output = []
        for pos in range(Image.shape[0]):
            
            out = np.zeros((out_x,out_y))

            for i in range(out_x):
                for j in range(out_y):
                    out[i,j] = np.mean(Image[0][i*s:i*s+x_k,j*s:j*s+y_k])
            output = output + [out]
            
        output = np.array(output)
        
        return output
    
    def reverse(self,Image,Errors):
        if(len(Image.shape)==2):
            Image=np.reshape(Image,(1,Image.shape[0],Image.shape[1]))
        
        x,y = Image[0].shape
        x_k,y_k = self.shape
        err_x,err_y = Errors[0].shape
            
        output = []
        for pos in range(Image.shape[0]):

            out = np.zeros_like(Image[pos],Errors.dtype)

            for i in range(err_x):
                for j in range(err_y):
                    out[i:i+x_k,j:j+y_k] += Errors[pos,i,j]/(x_k*y_k)
            output = output + [out]
        output = np.array(output)
        return output
    
# Flatten
class flatten():
    
    def forward(self,Image):
        output = np.reshape(Image,np.product(Image.shape))
        return output
    
    def reverse(self,Image,errors):
        output = np.reshape(errors,Image.shape)
        return output

# Dense
class fully_connected():
    def __init__(self,input_size,n):
        
        self.w = np.random.rand(n,input_size)/(input_size+1)
        self.b = np.random.rand(n)/(input_size+1)
        self.n = n
    
    def forward(self,Input):
        output = []
        for layer in range(self.n):
            out = np.dot(Input,self.w[layer]) + self.b[layer]
            output = output + [out]
        output = np.array(output)
        return output
    
    def reverse(self,Input,errors):
        
        output = np.zeros_like(Input)
        for layer in range(self.n):
            self.w[layer] = self.w[layer] - errors[layer]*Input
            self.b[layer] = self.b[layer] - errors[layer]
            output = output + errors[layer]*self.w[layer]   
        output = np.array(output)
        return output
    
# softmax
class softmax():
    
    def forward(self,Input):

        Numerator = np.exp(Input)
        output = Numerator/sum(Numerator)
        
        return (output)
    
    def reverse(self,Input,errors):
            
        Soft = self.forward(Input)
        
        Soft_list = np.array([list(Soft)] * len(Soft))
        derivative = Soft_list.T*(np.identity(len(Soft)) - Soft_list)
        
        output = np.matmul(errors,derivative)
        
        return output

def SoftmaxLoss(truth,predicted):
    
    loss = -1* np.sum(truth*np.log(predicted))
    
    Numerator = np.exp(predicted)
    softmax = Numerator/sum(Numerator)
        
    grad = softmax
    grad[truth == 1] = softmax[truth == 1] - 1
    
    return (loss,grad)
# Must work for both adam as well as SGD algos

In [ ]:
temp =  conv2d_layer((2,5,5),2)
Img = np.array([
    [
    [1,2,3,3,2],
    [1,3,5,2,3],
    [2,5,3,1,2],
    [3,1,2,1,2],
    [5,2,4,3,1]],
    [
    [1,2,3,3,2],
    [1,3,5,2,3],
    [2,5,3,1,2],
    [3,1,2,1,2],
    [5,2,4,3,1]]
])

temp.forward(Img)

In [ ]:
Model = [] #Input = 1*32*32

#Layer 0
Model = Model + [conv2d_layer((1,5,5),6)] #Out = 6*28*28

#Layer 1
Model = Model + [tanh()] #Out = 6*28*28

#Layer 2
Model = Model + [AveragePooling2D((2,2),2)] # Out = 6*14*14

#Layer 3
Model = Model + [conv2d_layer((6,5,5),16)] #Out = 16*10*10

#Layer 4
Model = Model + [tanh()] #Out = 16*10*10

#Layer 5
Model = Model + [AveragePooling2D((2,2),2)]# Out = 16*5*5

#Layer 6
Model = Model + [conv2d_layer((16,5,5),120)] #Out = 120*1*1

#Layer 7
Model = Model + [tanh()] #Out = 120*1*1

#Layer 8
Model = Model + [flatten()] #Out = 120

#Layer 9
Model = Model + [fully_connected(120,84)] #Out = 84

#Layer 10
Model = Model + [tanh()] #Out = 84

#Layer 11
Model = Model + [fully_connected(84,10)] #Out = 10

#Layer 12
Model = Model + [softmax()] #Out = 10

In [ ]:
### Read dataset
from sklearn.datasets import load_digits
data = load_digits()
list(data.target_names)

In [ ]:
from sklearn.model_selection import train_test_split
x = data['images']/16
y = data['target']
col_names = data['target_names']
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size =0.2)

In [ ]:
epochs = 5
Recorded_loss = []
Learning_rate = 0.01

x_train = x_train[:50]
y_train = y_train[:50]

n_layers = len(Model)
for ep in tqdm.tqdm(range(epochs),leave=True,desc = "Epochs"):
    
    Data = tqdm.tqdm(zip(x_train,y_train),total = len(y_train),leave=False,desc = "Images")
    
    for Input_img,truth in Data:
        
        truth_onehot = np.array([0]*10)
        truth_onehot[truth] = 1
        
        outputs = [cv2.resize(Input_img,(32,32))]

        for i in range(n_layers):
            out = Model[i].forward(outputs[-1])
#             print(i,out.shape)
#             print(out)
            outputs += [out.copy()]
            
    
        Loss, error = SoftmaxLoss(truth_onehot,outputs[-1])
#         print(Loss,error)
        error = error * Learning_rate
        Recorded_loss += [Loss] 
        
        Data.set_description("Current loss is: {}".format(np.round(Loss,5)))
        Data.refresh()
        
#         print("\n\nTruth:\n",truth_onehot,"\nPrediction:\n",outputs[-1])
#         print("Epoch: ",ep ,"\nLoss:",Loss)
        
        error = [error]
        for i in range(n_layers-1,-1,-1):
#             print("Reversing layer {}.".format(i))
            out = Model[i].reverse(outputs[i],error[-1])
            error += [out.copy()]
            
#         error.reverse()
#         for i in range(len(error)): 
#             print("\n*******************Output of layer {} is:*******************\n\n".format(i-1),outputs[i])
#             print("\n*******************Gradient entering layer {} output:*******************\n\n".format(i-1),error[i])
    
    Data.set_description("Completed")
    Data.refresh()

In [ ]:
plt.plot(Recorded_loss)
plt.title("Error per image ('Stochastic')")

val = 100
Avg_loss = []
for i in range(len(Recorded_loss)-val):
    Avg_loss +=  [np.mean(Recorded_loss[i:i+val])]

plt.figure()
plt.title("Loss after smoothing")
plt.plot(Avg_loss)

In [ ]:
#Display confusion Matrix

prediction = []
test_set = x_test[0:50]

for Input_img in tqdm.tqdm(test_set,total = len(test_set)):
    outputs = [cv2.resize(Input_img,(32,32))]
    for i in range(n_layers):
        out = Model[i].forward(outputs[-1])
        outputs += [out.copy()]
    prediction += [np.argmax(out)]
    
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(prediction, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

###### Keras Implementation

In [ ]:
size = (32, 32)
x_train = np.array([cv2.resize(x_train[i],size) for i in range(x_train.shape[0])])
x_train = np.reshape(x_train, (len(x_train), 32, 32, 1))
x_test = np.array([cv2.resize(x_test[i],size) for i in range(x_test.shape[0])])
x_test = np.reshape(x_test, (len(x_test), 32, 32, 1))

In [ ]:
y_train = pd.get_dummies(y_train).values
y_test_val = pd.get_dummies(y_test).values

In [ ]:
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(5,5), padding='same', activation='tanh', input_shape=(32, 32, 1)))
model.add(AvgPool2D(strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))
model.add(AvgPool2D(strides=2))
model.add(Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='tanh'))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(Dense(10, activation='softmax'))

In [ ]:
adam = Adam(learning_rate=5e-4)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

In [ ]:
model.fit(x= x_train,y= y_train, batch_size=1, epochs=epochs, verbose=0,callbacks=[TqdmCallback(verbose=1)])

In [ ]:
predicted = model.predict(x_test)
prediction = np.argmax(predicted, axis=1)

In [ ]:
#Display confusion Matrix
    
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(prediction, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

##### Q.3. Build CNN(LeNet5) model from scratch to recognize diabetes-fromPima-Indians-diabetesdatabase (i.e. https://github.com/duonghuuphuc/keras/tree/master/dataset ). Use Adamgradient descent algorithm to learn model with parameters for α = 0.01 and random parameters of theCNN model for Binary cross entropy loss function. 
- (i) Visualize input dataset and Plot comparative loss curve for at least 200 epochs. 
- (ii) Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy on test datsetand ROC curve for each loss function. 
- (iii) Implement above CNN model with Keras/Tensorflow/Pytorch Library and verify the above results.

In [ ]:
def BinaryCrossEntropy(y_truth,y_pred):
    Loss = -1* (y_truth*np.log(y_pred) + (1-y_truth)*np.log(1-y_pred))
    grad = -1* ((y_truth/y_pred) - (1-y_truth)/(1-y_pred))
    return (Loss,grad)

class sigmoid:
    
    def forward(self,Input):
        output = 1/(1+np.exp(-1*Input))
        return output
    
    def reverse(self,Input,Errors):
        sig = self.forward(Input)
        output = sig*(1-sig)*Errors
        return output

In [ ]:
#Load and display dataset

path = "Data/"
data = pd.read_csv(path+'pima-indians-diabetes.csv',header=None)
print(data.info())
data.head()

In [ ]:
#Split dataset

x = np.array(data.drop(8,axis = 1))
y = np.array(data[8])

x = x/np.max(x,axis = 0)

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size =0.2)

In [ ]:
Model = [] #Input = 1*32*32

#Layer 0
Model = Model + [fully_connected(8,30)] #Out = 84

#Layer 1
Model = Model + [tanh()] #Out = 84

#Layer 2
Model = Model + [fully_connected(30,10)] #Out = 10

#Layer 3
Model = Model + [tanh()] #Out = 84

#Layer 4
Model = Model + [fully_connected(10,1)] #Out = 10

#Layer 5
Model = Model + [sigmoid()] #Out = 84

In [ ]:
epochs = 15
Recorded_loss = []
Learning_rate = 0.01


n_layers = len(Model)
for ep in tqdm.tqdm(range(epochs),leave=True,desc = "Epochs"):
    
    Data = tqdm.tqdm(zip(x_train,y_train),total = len(y_train),leave=False,desc = "Images")
    
    for Input_img,truth in Data:
        
        outputs = [Input_img]

        for i in range(n_layers):
            out = Model[i].forward(outputs[-1])
#             print(i,out.shape)
#             print(out)
            outputs += [out.copy()]
            
    
        Loss, error = BinaryCrossEntropy(truth,outputs[-1][0])
#         print(Loss,error)
        error = error * Learning_rate
        Recorded_loss += [Loss] 
        
        Data.set_description("Current loss is: {}".format(np.round(Loss,5)))
        Data.refresh()
        
#         print("\n\nTruth:\n",truth_onehot,"\nPrediction:\n",outputs[-1])
#         print("Epoch: ",ep ,"\nLoss:",Loss)
        
        error = [error]
        for i in range(n_layers-1,-1,-1):
#             print("Reversing layer {}.".format(i))
            out = Model[i].reverse(outputs[i],error[-1])
            error += [out.copy()]
            
#         error.reverse()
#         for i,layer in enumerate(error): 
#             print("\n*******************Output of layer {} is:*******************\n\n".format(i-1),outputs[i])
#             print("\n*******************Error in layer {} output:*******************\n\n".format(i-1),layer)
    
    Data.set_description("Completed")
    Data.refresh()

In [ ]:
plt.plot(Recorded_loss)
plt.title("Error per image ('Stochastic')")

val = 1200
Avg_loss = []
for i in range(len(Recorded_loss)-val):
    Avg_loss +=  [np.mean(Recorded_loss[i:i+val])]

plt.figure()
plt.title("Loss after smoothing")
plt.plot(Avg_loss)

In [ ]:
#Display confusion Matrix

prediction = []
test_set = x_test[0:50]

for Input_img in tqdm.tqdm(test_set,total = len(test_set)):
    outputs = [Input_img]
    for i in range(n_layers):
        out = Model[i].forward(outputs[-1])
        outputs += [out.copy()]
    prediction += [np.argmax(out)]
    
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(prediction, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

###### Keras Implementation

In [ ]:
model_2 = Sequential()
model_2.add(Dense(50, activation='tanh', input_shape=(8,)))
model_2.add(Dense(40, activation='tanh'))
model_2.add(Dense(30, activation='tanh'))
model_2.add(Dense(20, activation='tanh'))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(1, activation='sigmoid'))

In [ ]:
model_2.build()
model_2.summary()

In [ ]:
model_2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=adam)

In [ ]:
model_2.fit(x= x_train,y= y_train, batch_size=1, epochs=epochs, verbose=0,callbacks=[TqdmCallback(verbose=1)])

In [ ]:
predicted = model_2.predict(x_test)
prediction = np.argmax(predicted, axis=1)

In [ ]:
#Display confusion Matrix
    
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(prediction, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

##### Q. 4. Build a CNNmodel from scratch to recognize human emotion using Facial emotionrecognition dataset (FER2013) (https://github.com/gitshanks/fer2013)(split dataset into training dataset (80%) and testing dataset (20%) ). For implementation, use a Adam gradient descent algorithm to learn model with parameters for α = 0.01 and random parameters of the CNN model forthe Softmax loss function 
- (i)Visualize Facial emotion recognition dataset (FER2013). 
- (ii) Plot a comparative loss curve for at least 200 epochs. 
- (iii)Print confusion matrix, calculate classification metrics such as precision, recall, f1-score and accuracy on test dataset and ROC curve 
- (iv) Implement above CNN model withKeras/Tensorflow/Pytorch Library and verify the above results.

In [ ]:
#Reading and viewing data

path = "Data/"
data = pd.read_csv(path+'icml_face_data.csv')

print("List of emotions: ",data['emotion'].unique())
data.head()

In [ ]:
#Function to prepare data
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

#Prepare data
x,y = prepare_data(data)
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size =0.2)

print(np.shape(x_train),np.shape(y_train))

In [ ]:
#Visualize Facial emotion recognition dataset

plt.figure()
fig, ax = plt.subplots()

for i in range(6):
    plt.subplot(2,3,i+1)
    t= np.random.randint(28709)
    plt.title("Emotion: {}".format(y_train[t]))
    plt.imshow(x_train[t])

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);

In [ ]:
Model = [] #Input = 1*32*32

#Layer 0
Model = Model + [conv2d_layer((1,5,5),6)] #Out = 6*28*28

#Layer 1
Model = Model + [tanh()] #Out = 6*28*28

#Layer 2
Model = Model + [AveragePooling2D((2,2),2)] # Out = 6*14*14

#Layer 3
Model = Model + [conv2d_layer((6,5,5),16)] #Out = 16*10*10

#Layer 4
Model = Model + [tanh()] #Out = 16*10*10

#Layer 5
Model = Model + [AveragePooling2D((2,2),2)]# Out = 16*5*5

#Layer 6
Model = Model + [conv2d_layer((16,5,5),120)] #Out = 120*1*1

#Layer 7
Model = Model + [tanh()] #Out = 120*1*1

#Layer 8
Model = Model + [flatten()] #Out = 120

#Layer 9
Model = Model + [fully_connected(120,84)] #Out = 84

#Layer 10
Model = Model + [tanh()] #Out = 84

#Layer 11
Model = Model + [fully_connected(84,7)] #Out = 10

#Layer 12
Model = Model + [softmax()] #Out = 10

In [ ]:
epochs = 5
Recorded_loss = []
Learning_rate = 0.01

x_train = x_train[:50]
y_train = y_train[:50]

n_layers = len(Model)
for ep in tqdm.tqdm(range(epochs),leave=True,desc = "Epochs"):
    
    Data = tqdm.tqdm(zip(x_train,y_train),total = len(y_train),leave=False,desc = "Images")
    
    for Input_img,truth in Data:
        
        truth_onehot = np.array([0]*7)
        truth_onehot[truth] = 1
        
        outputs = [cv2.resize(Input_img,(32,32))]

        for i in range(n_layers):
            out = Model[i].forward(outputs[-1])
#             print(i,out.shape)
#             print(out)
            outputs += [out.copy()]
            
    
        Loss, error = SoftmaxLoss(truth_onehot,outputs[-1])
#         print(Loss,error)
        error = error * Learning_rate
        Recorded_loss += [Loss] 
        
        Data.set_description("Current loss is: {}".format(np.round(Loss,5)))
        Data.refresh()
        
#         print("\n\nTruth:\n",truth_onehot,"\nPrediction:\n",outputs[-1])
#         print("Epoch: ",ep ,"\nLoss:",Loss)
        
        error = [error]
        for i in range(n_layers-1,-1,-1):
#             print("Reversing layer {}.".format(i))
            out = Model[i].reverse(outputs[i],error[-1])
            error += [out.copy()]
            
        error.reverse()
#         for i,layer in enumerate(error): 
#             print("\n*******************Output of layer {} is:*******************\n\n".format(i-1),outputs[i])
#             print("\n*******************Error in layer {} output:*******************\n\n".format(i-1),layer)
    
    Data.set_description("Completed")
    Data.refresh()

In [ ]:
plt.plot(Recorded_loss)
plt.title("Error per image ('Stochastic')")

val = 100
Avg_loss = []
for i in range(len(Recorded_loss)-val):
    Avg_loss +=  [np.mean(Recorded_loss[i:i+val])]

plt.figure()
plt.title("Loss after smoothing")
plt.plot(Avg_loss)

In [ ]:
#Display confusion Matrix

prediction = []
test_set = x_test[0:50]

for Input_img in tqdm.tqdm(test_set,total = len(test_set)):
    outputs = [cv2.resize(Input_img,(32,32))]
    for i in range(n_layers):
        out = Model[i].forward(outputs[-1])
        outputs += [out.copy()]
    prediction += [np.argmax(out)]
    
y_actu = pd.Series(y_test, name='Actual')
y_pred = pd.Series(prediction, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion